In [ ]:
import os
import cv2

Connecting Google Drive to Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Loading the Darknet folder

In [ ]:
!unzip "/content/drive/My Drive/--path to zipped darknet folder--"

In [ ]:
#!cd darknet && sed '1 s/^.*$/GPU=0/; 2 s/^.*$/CUDNN=0/; 3 s/^.*$/OPENCV=1/' -i Makefile

Compiling Darknet

In [ ]:
%cd /content/darknet-master
!make
!chmod +x ./darknet

Saving weights during training

In [ ]:
!rm /content/darknet/backup -r
!ln -s /content/drive/'My Drive'/"--path to folder for saving weights--"

Changing files to unix format

In [ ]:
!sudo apt install dos2unix

In [ ]:
!dos2unix ./data/train.txt
!dos2unix ./data/val.txt
!dos2unix ./data/yolo.data
!dos2unix ./data/yolo.names
!dos2unix ./cfg/yolov3_custom_test.cfg

# Training

In [ ]:
%cd /content/darknet
!./darknet detector train data/yolo.data cfg/yolov3_custom_train.cfg darknet53.conv.74 | tee train_log.txt

In [ ]:
!nvidia-smi

In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns

iters = []
losses = []
total_losses = []
with open("train_log.txt", 'r') as f:
    for i,line in enumerate(f):
        if "images" in line:
            iters.append(int(line.strip().split()[0].split(":")[0]))
            losses.append(float(line.strip().split()[2]))        
            total_losses.append(float(line.strip().split()[1].split(',')[0]))

plt.figure(figsize=(20, 5))
plt.subplot(1,2,1)
sns.lineplot(iters, total_losses, label="totla loss")
sns.lineplot(iters, losses, label="avg loss")
plt.xlabel("Iteration")
plt.ylabel("Loss")

plt.subplot(1,2,2)
sns.lineplot(iters, total_losses, label="totla loss")
sns.lineplot(iters, losses, label="avg loss")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.ylim([0, 4.05])

Showing some predictions

In [ ]:
def get_bbox(patient_id):
  bbox = []
  with open("detection.txt", 'r') as f:
      for i,line in enumerate(f):
          if "BBox-LTRB" in line:
              bb = (line.strip('\n')).split(":")[1]
              bbox = bbox + bb.split(",")
  if len(bbox) > 5:
    l1 = int(bbox[0])
    t1 = int(bbox[1])
    r1 = int(bbox[2])
    b1 = int(bbox[3])
    l2 = int(bbox[4])
    t2 = int(bbox[5])
    r2 = int(bbox[6])
    b2 = int(bbox[7])
    rect1 = cv2.rectangle(cv2.imread("test_imgs/{}.png".format(patient_id)), (l1, t1), (r1, b1), (0,117,97), 2)
    rect2 = cv2.rectangle(rect1, (l2, t2), (r2, b2), (0,117,97), 2)
    plt.figure()
    plt.imshow(rect2)
    plt.axis('off')
    plt.title('YOLOv3')
  elif len(bbox) == 0:
    plt.figure()
    plt.imshow(cv2.imread("test_imgs/{}.png".format(patient_id)))
    plt.axis('off')
    plt.title('YOLOv3')
  else:
    l = int(bbox[0])
    t = int(bbox[1])
    r = int(bbox[2])
    b = int(bbox[3])        
    rect = cv2.rectangle(cv2.imread("test_imgs/{}.png".format(patient_id)), (l, t), (r, b), (0,117,97), 2)
    plt.figure()
    plt.imshow(rect)
    plt.axis('off')
    plt.title('YOLOv3')

def get_ltrb(xc,yc,rw,rh):
    l = int((xc-rw/2)*128)
    t = int((yc-rh/2)*128+rh*128)
    r = int(l + rw*128)
    b = int((yc-rh/2)*128)
    return [l,t,r,b]

def get_bbox_0(patient_id):
  bb = []
  path = "test_imgs/locs/{}.txt".format(patient_id)
  if os.path.isfile(path):
    with open(path, 'r') as f:
      for line in f:
        bb = bb + [float(i) for i in line.split()[1:5]]
    if len(bb) > 5:
      xc1 = bb[0]
      yc1 = bb[1]
      rw1 = bb[2]
      rh1 = bb[3]
      xc2 = bb[4]
      yc2 = bb[5]
      rw2 = bb[6]
      rh2 = bb[7]
      ltrb1 = get_ltrb(xc1,yc1,rw1,rh1)
      ltrb2 = get_ltrb(xc2,yc2,rw2,rh2)
      rect1 = cv2.rectangle(cv2.imread("test_imgs/{}.png".format(patient_id)), (ltrb1[0], ltrb1[1]), (ltrb1[2], ltrb1[3]), (143,19,97), 2)
      rect12 = cv2.rectangle(rect1, (ltrb2[0], ltrb2[1]), (ltrb2[2], ltrb2[3]), (143,19,97), 2)
      plt.subplot(1,2,1)
      plt.imshow(rect2) 
      plt.axis('off')
      plt.title('Radiolozi')
    else:      
      xc = bb[0]
      yc = bb[1]
      rw = bb[2]
      rh = bb[3]
      ltrb = get_ltrb(xc,yc,rw,rh)
      rect = cv2.rectangle(cv2.imread("test_imgs/{}.png".format(patient_id)), (ltrb[0], ltrb[1]), (ltrb[2], ltrb[3]), (143,19,97), 2)
      plt.subplot(1,2,1)
      plt.imshow(rect)   
      plt.axis('off')
      plt.title('Radiolozi') 
  else:
    plt.subplot(1,2,1)      
    plt.imshow(cv2.imread("test_imgs/{}.png".format(patient_id)))   
    plt.axis('off')
    plt.title('Radiolozi')

In [ ]:
cfg_path = "cfg/yolov3_custom_test.cfg"
weights_path = "backup/yolov3_custom_train_2000.weights"
net = cv2.dnn.readNetFromDarknet(cfg_path, weights_path)

In [ ]:
def predict(image, pat_id):
    ln = net.getLayerNames()
    ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    
    blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)

    net.setInput(blob)
    layerOutputs = net.forward(ln)
    
    boxes = []
    confidences = []
    classIDs = []
    threshold = 0.1
    br_det = 0
    ltrb = []
    for output in layerOutputs:
        for detection in output:
            if detection[4] > threshold:
              br_det = br_det + 1
              x, y, w, h = detection[:4]
              ltrb = ltrb + [get_ltrb(x,y,w,h)]
    if br_det == 0:
      plt.figure()
      get_bbox_0(pat_id)
      plt.subplot(1,2,2)
      plt.imshow(image)
      plt.title('YOLOv3')
      plt.axis('off')
      
    elif br_det > 0:
      rect = image
      ltrb = ltrb[0:2]
      for l in ltrb:
        rect = cv2.rectangle(rect, (l[0], l[1]), (l[2], l[3]), (0,117,97), 2)
      plt.figure()
      get_bbox_0(pat_id)
      plt.subplot(1,2,2)
      plt.imshow(rect)
      plt.title('YOLOv3')
      plt.axis('off')
    return

In [ ]:
imgs = os.listdir("test_imgs")
for img in imgs:
  if img != 'locs':
    predict(cv2.imread(os.path.join("test_imgs",img)),img.split('.')[0])
